In [301]:
# import modules
import numpy as np
import pandas as pd
import os
from scipy.linalg import lstsq # To increase efficiency, and double check the results
import sympy as sp
import re
np.set_printoptions(precision=5,linewidth=100) # adjust setting for prettier printing


In [246]:
working_folder='E:/Ender/coursework/photogrammetry/Ass2' # set the folder storing the files
os.chdir(working_folder) # change the path to working_folder
# Read .prj file to get the max iteration times and threshold for stopping the iteration
with open('GrassHoper.prj','r') as f:
    txt=f.read() 
    txt=np.array(re.split('>>|\ |\n',txt)) # split the text so that I can index the value.
max_iteration=np.int32(txt[-7]) # Max iteration times
maxSigma=np.float32(txt[-6]) # The maximum allowable difference between unit weight variances in successive iterations. If the difference is less than this value, then the iterations will stop.
print('max_iteration:', max_iteration)
print('maxSigma:', maxSigma)

# Read .icf to get xy (image coordinates)
xy_img=pd.read_csv('OUT_OutlierFree_mm-9_17.icf',sep='\t|\   |\  ',engine='python',names=['Image_ID','Point_ID','x','y','w1','w2','w3','w4'])
## Subset the xy for each image
xy_img_09=xy_img[xy_img['Image_ID']=='65_09_20180803.jpg'] # subset the dataframe to get xy for 65_09_20180803.jpg
xy_img_10=xy_img[xy_img['Image_ID']=='65_10_20180803.jpg'] # repeat the process for the other images.
xy_img_14=xy_img[xy_img['Image_ID']=='65_14_20180803.jpg']
xy_img_18=xy_img[xy_img['Image_ID']=='65_18_20180803.jpg']
# Read IOP based on the output from lab 1
with open('lab2.cam','r') as f:
    txt=f.read() 
    txt=np.array(re.split('\t|\n',txt)) # split the text so that I can index the value.
xp,yp,c=np.float32(txt[9:12]) # extract xp,yp,c

print(f'xp={xp}, yp={yp}, c={c}')

k1,k2,k3,p1,p2=np.float32(txt[22:27]) #extract distortion parameters
print(f'k1={k1}, k2={k2}, k3={k3}, p1={p1}, p2={p2}')
# Read GCP from .gcp
gcp=pd.read_csv('NewTestField_gcp.gcp',engine='python',sep='\t|\   ',names=['Point_ID','X','Y','Z'],usecols=[0,1,2,3])
X_gcp=np.array(gcp['X'])
Y_gcp=np.array(gcp['Y'])
Z_gcp=np.array(gcp['Z'])
# gcp.head(5)
# Read .ori as initial guess for EOP for image9,10,14,18
with open('EOPs.ori','r') as f:
    line=0
    for i in f:
        line=line+1 # count line number
        text=f.readline()
        if line==9: # image 9
            text=np.array(re.split('\ ',text)) # divide numbers to a list
            omg_09,phi_09,kpp_09,Xo_09,Yo_09,Zo_09=(text[1:7])#extract EOP (initial guess)
        if line == 10:
            text=np.array(re.split('\ ',text)) # divide numbers to a list
            omg_10,phi_10,kpp_10,Xo_10,Yo_10,Zo_10=(text[1:7])#extract EOP (initial guess)
        if line == 14:
            text=np.array(re.split('\ ',text)) # divide numbers to a list
            omg_14,phi_14,kpp_14,Xo_14,Yo_14,Zo_14=(text[1:7])#extract EOP (initial guess)
        if line ==18:
            text=np.array(re.split('\ ',text)) # divide numbers to a list
            omg_18,phi_18,kpp_18,Xo_18,Yo_18,Zo_18=(text[1:7])#extract EOP (initial guess)

# print the value for checking 
print(omg_09,phi_09,kpp_09,Xo_09,Yo_09,Zo_09)
print(omg_10,phi_10,kpp_10,Xo_10,Yo_10,Zo_10)
print(omg_14,phi_14,kpp_14,Xo_14,Yo_14,Zo_14)
print(omg_18,phi_18,kpp_18,Xo_18,Yo_18,Zo_18)


max_iteration: 200
maxSigma: 1e-12
xp=0.06722940504550934, yp=-0.11779332906007767, c=8.15868854522705
k1=-0.0002753081498667598, k2=8.035393875616137e-06, k3=-2.082014987081493e-07, p1=6.32564042462036e-05, p2=-7.169081800384447e-05
-5.739377 -0.248099 2.050632 2.2 3.5 3.22
-1.055093 -5.256979 92.014892 1.636114 2.184056 3.727135
-8.214459 -17.492066 90.607889 0.938012 2.492823 2.988345
12.919527 0.159243 90.221535 1.981303 1.155948 3.933779


In [190]:
# angle conversion
def deg2rad(angle): return np.deg2rad(np.float32(angle)) # When I read the angles from the text files, they are string, so I need to convert the type first.
def rad2deg(angle): return np.rad2deg(angle) # convert radian to degree
#Set up symbolic function
s_omg,s_phi,s_kpp,s_x,s_y,s_X,s_Y,s_Z,s_X0,s_Y0,s_Z0=sp.symbols('s_omg s_phi s_kpp s_x s_y s_X s_Y s_Z s_X0 s_Y0 s_Z0')
R_omg = sp.Matrix([[1,0,0],[0,sp.cos(s_omg),-sp.sin(s_omg)],[0,sp.sin(s_omg),sp.cos(s_omg)]]) # rotation matrix for omega
R_phi=sp.Matrix([[sp.cos(s_phi),0,sp.sin(s_phi)],[0,1,0],[-sp.sin(s_phi),0,sp.cos(s_phi)]]) # rotation matrix for phi
R_kpp=sp.Matrix([[sp.cos(s_kpp),-sp.sin(s_kpp),0],[sp.sin(s_kpp),sp.cos(s_kpp),0],[0,0,1]]) # rotation matrix for kappa
s_R=R_omg*R_phi*R_kpp # Final rotation matrix 
s_Nx=s_R[0,0]*(s_X-s_X0)+s_R[1,0]*(s_Y-s_Y0)+s_R[2,0]*(s_Z-s_Z0) # define Nx
s_Ny=s_R[0,1]*(s_X-s_X0)+s_R[1,1]*(s_Y-s_Y0)+s_R[2,1]*(s_Z-s_Z0) # define Ny
s_D =s_R[0,2]*(s_X-s_X0)+s_R[1,2]*(s_Y-s_Y0)+s_R[2,2]*(s_Z-s_Z0) # define D
sym_func=sp.Matrix([[s_x-xp+c*s_Nx/s_D],[s_y-yp+c*s_Ny/s_D]]) # define the symbolic functions
sym_A_ls=sym_func.jacobian([s_X0,s_Y0,s_Z0,s_omg,s_phi,s_kpp]) # define the symbolic functions
eval_func=sp.lambdify((s_x,s_y,s_X,s_Y,s_Z,s_X0,s_Y0,s_Z0,s_omg,s_phi,s_kpp),sym_func,modules='numpy') # lambdify the symbolic function
eval_A_ls=sp.lambdify((s_X,s_Y,s_Z,s_X0,s_Y0,s_Z0,s_omg,s_phi,s_kpp),sym_A_ls,modules='numpy') # # lambdify the symbolic function

In [298]:
def preprocessing(xy_img):
    '''
    This function is used to pre-process the data for LSA
    '''
    num_observation=len(xy_img['x']) #compute number of observation
    xx=np.array(xy_img['x']).astype(np.float32) # define x_img
    yy=np.array(xy_img['y']).astype(np.float32) # define y_img
    point_ID=np.array(xy_img['Point_ID']).astype(np.int16)
    dof=2*num_observation-6 # compute degree of freedom
    # The following lines compute the distortion
    x_bar=xx-xp
    y_bar=yy-yp
    rd=np.sqrt(x_bar**2+y_bar**2) # distance from principle point
    dist_x=x_bar*(k1*rd**2+k2*rd**4+k3*rd**6)+p1*(rd**2+2*x_bar**2)+2*p2*x_bar*y_bar
    dist_y=y_bar*(k1*rd**2+k2*rd**4+k3*rd**6)+p2*(rd**2+2*y_bar**2)+2*p1*x_bar*y_bar
    # the following lines compute x_corrected and y_corrected
    x_corrected=xx-dist_x 
    y_corrected=yy-dist_y
    return x_corrected,y_corrected,dof, num_observation,point_ID


In [281]:
def LSA(Xo,Yo,Zo,omega,phi,kappa,max_iteration,maxSigma,xy_img):
    Xo=np.float32(Xo)
    Yo=np.float32(Yo)
    Zo=np.float32(Zo)
    # preprocess data
    x_corrected,y_corrected,dof,num_observation,point_ID=preprocessing(xy_img)
    # convert the unit of angles
    omega=deg2rad(omega)
    phi=deg2rad(phi)
    kappa=deg2rad(kappa)
    # create a dictionary to store the EOP of each iteration
    x0_all=dict()
    keys=['X0 (mm)','Y0 (mm)','Z0 (mm)','omega (deg)','phi (deg)','kappa (deg)']
    for i in keys: 
        x0_all[i]=[] 
    #---------------------
    n_iter=0 # define number of iteration
    x0=np.array([Xo,Yo,Zo,omega,phi,kappa]) # set the initial guess
    idx_old=999999999 # set the initial idx with a huge value so that it won't stop at the first iteration

    while n_iter < max_iteration:
        # The following lines compute f and A_ls so that f= A_lsdX
        f=np.zeros((2*num_observation,1))
        A_ls=np.zeros((2*num_observation,6))
        for i in range(num_observation):
            n_gcp=point_ID[i]-1
            # compute values in A_ls
            A_ls[2*i:2*i+2]=eval_A_ls(X_gcp[n_gcp],Y_gcp[n_gcp],Z_gcp[n_gcp],x0[0],x0[1],x0[2],x0[3],x0[4],x0[5]) 
            # compute value in f
            f[2*i:2*i+2]=-eval_func(x_corrected[i],y_corrected[i],X_gcp[n_gcp],Y_gcp[n_gcp],Z_gcp[n_gcp],x0[0],x0[1],x0[2],x0[3],x0[4],x0[5])
        #-------------------------------------------------------------
        # compute best estimation
        dx,sigma_new,rank,s=lstsq(A_ls,f) # This function is more efficient
        # dx=np.linalg.inv(A_ls.T@A_ls)@A_ls.T@f # This line is equivalent to the previous line
        dx=dx.reshape(-1) # convert 2D array to 1D to avoid confusion to avoid bug caused by typo
        #-------------------------------------------------------------
        # update x0
        x0=x0+dx
        # record the updated EOP 
        for i in range(len(keys)):
            if keys[i]=='omega (deg)' or keys[i]=='phi (deg)' or keys[i]=='kappa (deg)':
                x0_all[keys[i]].append(rad2deg(x0[i]))
            else:
                x0_all[keys[i]].append(x0[i])
        #-------------------------------------------------------------
        # compute e
        f=f.reshape(-1)
        e=f-A_ls@dx
        # Assuming all the parameters have the same weights 
        idx=e@e.T # It should be e.T@e when e is nx1, but I use e in 1D (1xn). As a result, I need to use e@e.T 
        sigma_hat=idx/dof # compute sigma_hat
        print(n_iter,':',sigma_hat)
        # set up stopping condition
        if np.abs((idx-idx_old))<maxSigma:
            break
        ## update idx for stopping condition in next iteration
        idx_old=idx
        # update number of iteration
        n_iter=n_iter+1
    x0_all=pd.DataFrame(x0_all) # change the type of x0_all for better visualization
    # compute dispersion matrix
    D_hat=sigma_hat*np.linalg.inv(A_ls.T@A_ls)
    # compute std for EOP
    x0_std=np.sqrt(np.diag(D_hat))
    # divide residuals into x-residual and y-residual
    residual=e.reshape((len(x_corrected),2),order='C')
    return x0_all,D_hat,x0_std,residual


In [299]:

x0_all_09,D_hat_09,x0_std_09,residual_09=LSA(Xo_09,Yo_09,Zo_09,omg_09,phi_09,kpp_09,max_iteration,maxSigma,xy_img_09)
print('X0_std_09:',x0_std_09)
print('-------D_hat-------')
print(D_hat_09)
print('-----residual------')
residual_09=pd.DataFrame(residual_09, columns = ['x residual (mm)','y residual (mm)'])
display(residual_09.head(30))
x0_all_09


0 : 0.007258611952685998
1 : 0.0003989089566621661
2 : 0.0004139904595265831
3 : 0.00041612296375095984
4 : 0.00041612304772495145
5 : 0.00041612304772732476
X0_std_09: [0.01351 0.01218 0.00545 0.00295 0.00306 0.00125]
-------D_hat-------
[[ 1.82551e-04  2.81182e-08 -5.14911e-06  6.20948e-07  4.08964e-05  9.72750e-06]
 [ 2.81182e-08  1.48359e-04 -4.39914e-05 -3.54938e-05  4.64793e-08  7.74552e-07]
 [-5.14911e-06 -4.39914e-05  2.97117e-05  1.11912e-05 -1.38899e-06 -5.85095e-07]
 [ 6.20948e-07 -3.54938e-05  1.11912e-05  8.72371e-06  1.11099e-07 -6.07638e-08]
 [ 4.08964e-05  4.64793e-08 -1.38899e-06  1.11099e-07  9.35768e-06  2.12017e-06]
 [ 9.72750e-06  7.74552e-07 -5.85095e-07 -6.07638e-08  2.12017e-06  1.57281e-06]]
-----residual------


,x residual (mm),y residual (mm)
0,0.055067,-0.022115
1,0.002425,-0.018403
2,-0.010843,-0.011111
3,-0.008313,0.008738
4,0.054989,0.029605
5,-0.001160,0.010524
6,0.000198,-0.004801
7,-0.012866,-0.002938
8,-0.014848,-0.008364
9,0.017150,-0.014533


,X0 (mm),Y0 (mm),Z0 (mm),omega (deg),phi (deg),kappa (deg)
0,2.061088,2.764677,3.491400,-14.374644,-2.600690,2.491839
1,1.941590,3.005670,3.726410,-18.872484,-4.314689,1.659942
2,1.914001,3.062084,3.724552,-19.500790,-4.543810,1.458629
3,1.914152,3.061801,3.724021,-19.497086,-4.541278,1.456789
4,1.914148,3.061803,3.724020,-19.497115,-4.541328,1.456777
5,1.914148,3.061803,3.724020,-19.497114,-4.541328,1.456777


In [300]:

x0_all_14,D_hat_14,x0_std_14,residual_14=LSA(Xo_14,Yo_14,Zo_14,omg_14,phi_14,kpp_14,max_iteration,maxSigma,xy_img_14)
print('X0_std_09:',x0_std_14)
print('-------D_hat-------')
print(D_hat_14)
print('-----residual------')
residual_14=pd.DataFrame(residual_14, columns = ['x residual (mm)','y residual (mm)'])
display(residual_14.head(30))
x0_all_14

0 : 0.00039049947269565984
1 : 0.0003823648559148244
2 : 0.000382314700254953
3 : 0.0003823146980814439
4 : 0.00038231469808144527
X0_std_09: [0.00891 0.00874 0.00631 0.00258 0.00289 0.00128]
-------D_hat-------
[[ 7.93352e-05  3.78217e-06  3.89211e-05  1.19830e-06  2.50193e-05  1.01249e-06]
 [ 3.78217e-06  7.64427e-05 -1.81516e-05 -2.19530e-05 -1.41047e-07 -6.19555e-06]
 [ 3.89211e-05 -1.81516e-05  3.97801e-05  7.33578e-06  1.44397e-05  2.45734e-06]
 [ 1.19830e-06 -2.19530e-05  7.33578e-06  6.67832e-06  8.79837e-07  1.80750e-06]
 [ 2.50193e-05 -1.41047e-07  1.44397e-05  8.79837e-07  8.37188e-06  5.35339e-07]
 [ 1.01249e-06 -6.19555e-06  2.45734e-06  1.80750e-06  5.35339e-07  1.63856e-06]]
-----residual------


,x residual (mm),y residual (mm)
0,-0.033572,0.007889
1,0.003619,-0.007269
2,0.021210,-0.056343
3,-0.011754,-0.014735
4,-0.012768,0.012769
5,-0.007528,0.019689
6,-0.006973,-0.006089
7,0.034392,0.001318
8,0.017848,0.013548
9,0.011772,0.021121


,X0 (mm),Y0 (mm),Z0 (mm),omega (deg),phi (deg),kappa (deg)
0,1.022537,2.519151,2.792724,-8.508271,-17.640387,90.709885
1,1.018950,2.515723,2.805172,-8.466163,-17.606118,90.717942
2,1.018959,2.515656,2.805249,-8.465138,-17.605547,90.718264
3,1.018959,2.515656,2.805249,-8.465137,-17.605543,90.718264
4,1.018959,2.515656,2.805249,-8.465137,-17.605543,90.718264


In [303]:
x0_all_10,D_hat_10,x0_std_10,residual_10=LSA(Xo_10,Yo_10,Zo_10,omg_10,phi_10,kpp_10,max_iteration,maxSigma,xy_img_10)
print('X0_std_10:',x0_std_10)
print('-------D_hat-------')
print(D_hat_10)
print('-----residual------')
residual_10=pd.DataFrame(residual_10, columns = ['x residual (mm)','y residual (mm)'])
display(residual_10.head(30))
x0_all_10

0 : 0.0007492757733515534
1 : 0.0007581357269430365
2 : 0.0007582374386150168
3 : 0.0007582374408438055
4 : 0.0007582374408442086
X0_std_10: [0.01565 0.01533 0.00476 0.0038  0.00388 0.00131]
-------D_hat-------
[[ 2.45026e-04 -2.93374e-05  5.90317e-06  7.88256e-06  6.00055e-05 -1.26402e-06]
 [-2.93374e-05  2.34931e-04 -1.87352e-05 -5.75282e-05 -7.91119e-06  1.79597e-07]
 [ 5.90317e-06 -1.87352e-05  2.26869e-05  5.03161e-06  1.37571e-06 -3.68758e-08]
 [ 7.88256e-06 -5.75282e-05  5.03161e-06  1.44605e-05  2.12081e-06  8.20310e-08]
 [ 6.00055e-05 -7.91119e-06  1.37571e-06  2.12081e-06  1.50529e-05 -2.40549e-07]
 [-1.26402e-06  1.79597e-07 -3.68758e-08  8.20310e-08 -2.40549e-07  1.71525e-06]]
-----residual------


,x residual (mm),y residual (mm)
0,-0.038599,-0.032695
1,-0.030149,-0.007747
2,-0.008051,-0.001387
3,0.022521,-0.015375
4,0.050550,-0.081891
5,-0.013272,-0.019903
6,-0.013172,-0.007625
7,-0.016772,0.006151
8,-0.009807,0.010641
9,-0.002345,-0.025569


,X0 (mm),Y0 (mm),Z0 (mm),omega (deg),phi (deg),kappa (deg)
0,1.677296,2.209390,3.488569,-0.919388,-5.656132,92.057915
1,1.679588,2.210187,3.503848,-0.953987,-5.577833,92.058055
2,1.679598,2.210126,3.503926,-0.953032,-5.577757,92.058103
3,1.679597,2.210126,3.503926,-0.953039,-5.577761,92.058104
4,1.679597,2.210126,3.503926,-0.953039,-5.577761,92.058104


In [309]:
x0_all_14,D_hat_14,x0_std_14,residual_14=LSA(Xo_14,Yo_14,Zo_14,omg_14,phi_14,kpp_14,max_iteration,maxSigma,xy_img_14)
print('X0_std:',x0_std_14)
print('-------D_hat-------')
print(D_hat_14)
print('-----residual------')
residual_14=pd.DataFrame(residual_14, columns = ['x residual (mm)','y residual (mm)'])
display(residual_14.head(30))
x0_all_14
    

0 : 0.00039049947269565984
1 : 0.0003823648559148244
2 : 0.000382314700254953
3 : 0.0003823146980814439
4 : 0.00038231469808144527
X0_std: [0.00891 0.00874 0.00631 0.00258 0.00289 0.00128]
-------D_hat-------
[[ 7.93352e-05  3.78217e-06  3.89211e-05  1.19830e-06  2.50193e-05  1.01249e-06]
 [ 3.78217e-06  7.64427e-05 -1.81516e-05 -2.19530e-05 -1.41047e-07 -6.19555e-06]
 [ 3.89211e-05 -1.81516e-05  3.97801e-05  7.33578e-06  1.44397e-05  2.45734e-06]
 [ 1.19830e-06 -2.19530e-05  7.33578e-06  6.67832e-06  8.79837e-07  1.80750e-06]
 [ 2.50193e-05 -1.41047e-07  1.44397e-05  8.79837e-07  8.37188e-06  5.35339e-07]
 [ 1.01249e-06 -6.19555e-06  2.45734e-06  1.80750e-06  5.35339e-07  1.63856e-06]]
-----residual------


,x residual (mm),y residual (mm)
0,-0.033572,0.007889
1,0.003619,-0.007269
2,0.021210,-0.056343
3,-0.011754,-0.014735
4,-0.012768,0.012769
5,-0.007528,0.019689
6,-0.006973,-0.006089
7,0.034392,0.001318
8,0.017848,0.013548
9,0.011772,0.021121


,X0 (mm),Y0 (mm),Z0 (mm),omega (deg),phi (deg),kappa (deg)
0,1.022537,2.519151,2.792724,-8.508271,-17.640387,90.709885
1,1.018950,2.515723,2.805172,-8.466163,-17.606118,90.717942
2,1.018959,2.515656,2.805249,-8.465138,-17.605547,90.718264
3,1.018959,2.515656,2.805249,-8.465137,-17.605543,90.718264
4,1.018959,2.515656,2.805249,-8.465137,-17.605543,90.718264


In [307]:
x0_all_18,D_hat_18,x0_std_18,residual_18=LSA(Xo_18,Yo_18,Zo_18,omg_18,phi_18,kpp_18,max_iteration,maxSigma,xy_img_18)
print('X0_std:',x0_std_18)
print('-------D_hat-------')
print(D_hat_18)
print('-----residual------')
residual_18=pd.DataFrame(residual_18, columns = ['x residual (mm)','y residual (mm)'])
display(residual_18.head(30))
x0_all_18
    

0 : 0.0008581325919109502
1 : 0.0008536418409274642
2 : 0.0008537927423962262
3 : 0.0008537927130329473
4 : 0.0008537927130320911
X0_std: [0.01823 0.01777 0.00614 0.00424 0.00423 0.0016 ]
-------D_hat-------
[[ 3.32334e-04 -2.63297e-06 -2.11454e-05  1.11965e-06  7.62845e-05 -1.35597e-05]
 [-2.63297e-06  3.15760e-04  4.88758e-05 -7.44439e-05 -5.81505e-07  2.19359e-06]
 [-2.11454e-05  4.88758e-05  3.76998e-05 -1.20503e-05 -5.25310e-06  1.28061e-06]
 [ 1.11965e-06 -7.44439e-05 -1.20503e-05  1.79515e-05  2.65832e-07 -4.85028e-07]
 [ 7.62845e-05 -5.81505e-07 -5.25310e-06  2.65832e-07  1.79155e-05 -2.99223e-06]
 [-1.35597e-05  2.19359e-06  1.28061e-06 -4.85028e-07 -2.99223e-06  2.56809e-06]]
-----residual------


,x residual (mm),y residual (mm)
0,-0.037873,-0.036090
1,-0.027212,-0.015182
2,0.002292,-0.006498
3,0.040783,-0.012158
4,0.078112,-0.063122
5,-0.019608,-0.026071
6,-0.018038,-0.015229
7,-0.018220,-0.005470
8,-0.005199,0.005281
9,0.009530,-0.021891


,X0 (mm),Y0 (mm),Z0 (mm),omega (deg),phi (deg),kappa (deg)
0,2.099176,1.237009,3.693173,12.965394,0.802882,90.034425
1,2.090420,1.236958,3.707426,12.893191,0.769225,90.034486
2,2.090216,1.236784,3.707461,12.895477,0.766820,90.034598
3,2.090216,1.236782,3.707460,12.895498,0.766815,90.034601
4,2.090216,1.236782,3.707460,12.895498,0.766815,90.034601
